In [ ]:
import sys

sys.path[1:1] = ["_common", "_common/qiskit"]
sys.path[1:1] = ["../../_common", "../../_common/qiskit"]
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import time
import math
import numpy as np
np.random.seed(0)
import execute as ex
import metrics as metrics
from collections import defaultdict

In [ ]:
def HamiltonianSimulation(n_spins, K, t, w, h_x, h_z):
    '''
    Construct a Qiskit circuit for Hamiltonian Simulation
    :param n_spins:The number of spins to simulate
    :param K: The Trotterization order
    :param t: duration of simulation
    :return: return a Qiskit circuit for this Hamiltonian
    '''
    
    # allocate qubits
    qr = QuantumRegister(n_spins); cr = ClassicalRegister(n_spins); qc = QuantumCircuit(qr, cr, name="main")
    tau = t / K

    # start with initial state of 1010101...
    for k in range(0, n_spins, 2):
        qc.x(qr[k])
    qc.barrier()

    # loop over each trotter step, adding gates to the circuit defining the hamiltonian
    for k in range(K):
    
        # the Pauli spin vector product
        [qc.rx(2 * tau * w * h_x[i], qr[i]) for i in range(n_spins)]
        [qc.rz(2 * tau * w * h_z[i], qr[i]) for i in range(n_spins)]
        qc.barrier()
        
        # Use an optimal XXYYZZ combined operator
        # See equation 1 and Figure 6 in https://arxiv.org/pdf/quant-ph/0308006.pdf
        # optimized XX + YY + ZZ operator on each pair of qubits in linear chain
        for j in range(2):
            for i in range(j % 2, n_spins - 1, 2):
                qc.append(xxyyzz_opt_gate(tau).to_instruction(), [qr[i], qr[(i + 1) % n_spins]])

        qc.barrier()

    # measure all the qubits used in the circuit
    for i_qubit in range(n_spins):
        qc.measure(qr[i_qubit], cr[i_qubit])

    return qc

In [ ]:
# Optimal combined XXYYZZ gate (with double coupling) on q0 and q1 with angle 'tau'
def xxyyzz_opt_gate(tau):
    alpha = tau; beta = tau; gamma = tau
    qr = QuantumRegister(2); qc = QuantumCircuit(qr, name="xxyyzz_opt")
    qc.rz(3.1416/2, qr[1])
    qc.cx(qr[1], qr[0])
    qc.rz(3.1416*gamma - 3.1416/2, qr[0])
    qc.ry(3.1416/2 - 3.1416*alpha, qr[1])
    qc.cx(qr[0], qr[1])
    qc.ry(3.1416*beta - 3.1416/2, qr[1])
    qc.cx(qr[1], qr[0])
    qc.rz(-3.1416/2, qr[0])

    # save circuit example for display
    global XXYYZZ_    
    XXYYZZ_ = qc

    return qc

In [ ]:
import json
from qiskit import execute, Aer
backend = Aer.get_backend("qasm_simulator")

precalculated_data = {}

# parameters
w = 10
k = 3
t = 0.01

# store parameters in precalculated data
precalculated_data['w'] = w
precalculated_data['k'] = k
precalculated_data['t'] = t

# add parameter random values to precalculated data to ensure consistency
np.random.seed(26)
precalculated_data['h_x'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]
np.random.seed(75)
precalculated_data['h_z'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]

num_shots = 100000

for n_spins in range(2,21):
    
    h_x = precalculated_data['h_x'][:n_spins]
    h_z = precalculated_data['h_z'][:n_spins]

    qc = HamiltonianSimulation(n_spins, k, t, w, h_x, h_z)

    job = execute(qc, backend, shots=num_shots)
    result = job.result()

    counts = result.get_counts(qc)

    dist = {}
    for key in counts.keys():
        prob = counts[key] / num_shots
        dist[key] = prob

    # add dist values to precalculated data for use in fidelity calculation
    precalculated_data[f"Qubits - {n_spins}"] = dist

# https://stackoverflow.com/questions/36021332/how-to-prettyprint-human-readably-print-a-python-dict-in-json-format-double-q
with open('precalculated_data.json', 'w') as f:
    f.write(json.dumps(
        precalculated_data,
        sort_keys=True,
        indent=4,
        separators=(',', ': ')
        ))